In [9]:
#using Pkg
#Pkg.add("ProgressMeter")

In [1]:
using Distributions
using StatsBase
using Printf
using LinearAlgebra
using ProgressMeter
using Random

### Exercise 

1 . Generate data from model (1) with  𝑘=10  covariates.

In [2]:
Random.seed!(31415)
n = 2000 #number of observations
k = 10   #number of covariates
M = rand(Binomial(2,0.5),n,k)
M = M .- mean(M,dims=1)
X = [ones(n) M]

2000×11 Array{Float64,2}:
 1.0  -1.006   1.0125   0.0335   0.009  …   0.0155   0.009   0.0125   1.0025
 1.0  -0.006  -0.9875   1.0335   1.009     -0.9845   1.009   0.0125   0.0025
 1.0  -0.006   0.0125   0.0335   0.009      0.0155   0.009   0.0125   1.0025
 1.0  -0.006  -0.9875   0.0335   0.009     -0.9845   1.009   0.0125   0.0025
 1.0   0.994   0.0125   0.0335   0.009      0.0155   0.009   0.0125   1.0025
 1.0   0.994   1.0125   1.0335   1.009  …  -0.9845   0.009  -0.9875   0.0025
 1.0  -1.006   0.0125  -0.9665   0.009      1.0155  -0.991  -0.9875  -0.9975
 1.0  -1.006   0.0125  -0.9665   1.009     -0.9845   1.009  -0.9875   1.0025
 1.0  -0.006  -0.9875   0.0335   0.009      0.0155   0.009   0.0125  -0.9975
 1.0  -0.006   0.0125  -0.9665  -0.991     -0.9845   0.009   0.0125  -0.9975
 1.0  -0.006   0.0125   0.0335   0.009  …   0.0155   1.009   1.0125   1.0025
 1.0  -0.006   1.0125   1.0335   0.009     -0.9845   1.009   0.0125   1.0025
 1.0  -0.006   1.0125   0.0335  -0.991      0.0155

In [3]:
betaTrue = [1; randn(k)]
y = X*betaTrue+ randn(n);

2. Setup and solve the mixed model equations given by ([BLUPBeta]).

In [4]:
XPX = X'X
rhs = X'y
Vb = var(betaTrue[2:end])
Ve = 1
λ = Vb/Ve
mme = XPX
mme[2:end,2:end] += I*λ;

In [5]:
betaHat=inv(mme)*rhs

11-element Array{Float64,1}:
  0.9965642975700888 
 -0.23985017116825277
  1.3215375049478637 
 -0.6648765335226169 
  1.7164571072572692 
  0.3816503944688989 
 -0.31262561115510035
 -0.9973935168915621 
  0.19004467488805388
  0.3121677409885229 
  1.0578407001575203 

3. Sample the elements of  𝜷  using Gibbs.

In [6]:
function GibbsSample1(y,X,b,meanB,niter,λ)
    n,k = size(X)
     @showprogress "drawing MCMC samples..." for iter = 1:niter
        # sampling intercept
        w = y - X[:,2:k] * b[2:k]
        x = X[:,1]
        xpxi = 1/(x'x)
        bHat = (xpxi*x'w) 
        b[1] = rand(Normal(bHat, sqrt(xpxi))) # using residual var = 1     
        # sampling marker effects
        for i=2:k
            btemp = copy(b)
            btemp[i] = 0
            w = y - X*btemp
            x = X[:,i]
            xpxi = 1/((x'x) + λ)
            bHat = (xpxi*x'w)
            b[i] = rand(Normal(bHat, sqrt(xpxi))) # using residual var = 1  
        end 
        meanB += (b - meanB)/iter
        a[iter,:] = b
    end
    return meanB, a
end

GibbsSample1 (generic function with 1 method)

4. Compute the posterior mean of  𝜷  from the samples and compare with mixed model solutions.

### Compare with mixed model solution betaHat

In [8]:
niter = 100                    # number of samples
b     = zeros(k+1)
meanB = zeros(k+1)
a = zeros(niter,k+1)
@time meanB, a = GibbsSample1(y,X,b,meanB,niter,λ);

drawing MCMC samples...100%|████████████████████████████| Time: 0:00:00


  1.264396 seconds (3.06 M allocations: 217.957 MiB, 4.59% gc time)


In [9]:
[meanB betaHat]

11×2 Array{Float64,2}:
  0.993761   0.996564
 -0.240379  -0.23985 
  1.32206    1.32154 
 -0.66328   -0.664877
  1.72179    1.71646 
  0.384768   0.38165 
 -0.309759  -0.312626
 -0.99521   -0.997394
  0.195714   0.190045
  0.310968   0.312168
  1.05411    1.05784 

5. Compute the posterior covariance matrix from the sampled values. 


In [10]:
cov(a)

11×11 Array{Float64,2}:
  0.000651665  -8.42895e-5    2.48558e-5   …  -0.000128422  -2.76139e-5 
 -8.42895e-5    0.00146731   -3.90336e-6      -7.43539e-5   -5.49242e-5 
  2.48558e-5   -3.90336e-6    0.00103095       8.16413e-5    6.06216e-5 
 -5.07055e-5   -1.41525e-5    0.000158239     -9.00448e-5   -4.69847e-5 
 -0.000186333   0.000228324   0.000161735      0.000146325   0.000175542
  0.000131081  -6.62626e-5    1.58106e-5   …  -1.52908e-5   -0.000143815
 -8.13383e-5   -0.000129752   5.42766e-5      -5.11952e-6   -2.17382e-5 
 -4.41439e-5   -0.00014498    0.000125654      0.000165445  -1.13875e-5 
  3.85293e-5   -0.000212901   1.68746e-5      -0.000202583  -1.18488e-5 
 -0.000128422  -7.43539e-5    8.16413e-5       0.00123665    1.04021e-5 
 -2.76139e-5   -5.49242e-5    6.06216e-5   …   1.04021e-5    0.000992308

### compare with the inverse of mixed models

In [11]:
v=inv(mme)

11×11 Array{Float64,2}:
  0.0005        1.7034e-19  -2.14702e-20  …   1.02704e-19   1.345e-19  
  1.7034e-19    0.00101519  -1.95152e-5       2.20307e-5    1.09205e-6 
 -2.14702e-20  -1.95152e-5   0.000989398      1.28415e-5    4.99645e-7 
 -6.79513e-20  -1.35172e-6   2.179e-5        -3.72122e-6    1.16829e-5 
 -9.88436e-20   2.09381e-5   6.51383e-6      -9.56427e-8   -4.14459e-8 
 -9.59556e-20   9.35145e-6   1.52844e-5   …  -1.00062e-5   -3.25821e-5 
 -2.85862e-19  -1.52144e-5   5.46944e-5       1.19275e-5   -2.0614e-6  
  9.42679e-20  -4.93898e-5   2.25753e-5       3.39892e-5    5.72522e-6 
 -1.56183e-19  -4.14536e-5   2.83066e-6      -2.12451e-5   -2.96701e-6 
  1.02704e-19   2.20307e-5   1.28415e-5       0.00101552   -1.61024e-5 
  1.345e-19     1.09205e-6   4.99645e-7   …  -1.61024e-5    0.000974595